In [ ]:
# default_exp core
# default_cls_lvl 3

# Codestructure

1. Extract File Paths
2. Convert Paths to Sequence Objects (TensorSequence, TensorScalars, TensorSequence) <-> (input,input,output)
2. Data Manipulation
    1. Sequence pruning
    2. Sequence Resampling
4. Split Sequence Objects in Train, Validation
3. (Create Windows out of Sequence Objects)
5. Noise Injection in Training Dataset Input
5. Normalize Input 

In [ ]:
#export
from fastai2.data.all import *
import h5py

## 1. Extract File Paths
Der erste Schritt kann mit get_files von fastai2 erledigt werden. 

Alternativ kann dies mit weniger Code mit der get_hdf_files() Funktion erledigt werden.

In [ ]:
f_path = 'test_data/'
hdf_files = get_files(f_path,extensions='.hdf5',recurse=True)
len(hdf_files),hdf_files[0]

(3, PosixPath('test_data/train/Sim_RealisticCycle2.hdf5'))

In [ ]:
#export
hdf_extensions = ['.hdf5']
def get_hdf_files(path,recurse=True, folders=None):
    "Get hdf5 files in `path` recursively, only in `folders`, if specified."
    return get_files(path, extensions=hdf_extensions, recurse=recurse, folders=folders)

In [ ]:
hdf_files = get_hdf_files(f_path)
len(hdf_files),hdf_files[0]

(3, PosixPath('test_data/train/Sim_RealisticCycle2.hdf5'))

## 2. Convert Paths to Sequence Objects
Der Pfad wird unter Angabe der Spaltennamen in Sequenzen und Skalare Werte umgewandelt, um so am Ende ein 3-Tupel zu erhalten aus:
- (Sequence, Scalar, Sequence) <-> (input,input,output)

In [ ]:
#export
def hdf2sequence(hdf_path,c_names):
    with h5py.File(hdf_path,'r') as f:
#         import pdb; pdb.set_trace()
        l_array = [f[n][:][:,None] for n in c_names]
        seq = np.concatenate(l_array,axis=1)
        return seq

In [ ]:
hdf2sequence(hdf_files[0],['current','voltage']).shape

(265598, 2)

Die Funktion lässt sich mittels Pipeline auf eine Liste von Quellobjekten (hier Pfade) anwenden 

In [ ]:
pipe = Pipeline(partial(hdf2sequence,c_names=['current','voltage']))

In [ ]:
res_pipe = pipe(hdf_files)
len(res_pipe), res_pipe[0][0]

(3, array([0.       , 4.1873503], dtype=float32))

In [ ]:
#export
def hdf2scalars(hdf_path,c_names):
    with h5py.File(hdf_path,'r') as f:
#         import pdb; pdb.set_trace()
#         l_array = [f[n][:][:,None] for n in c_names]
#         seq = np.concatenate(l_array,axis=1)
        return None

### Tensor Tuple erstellen

In [ ]:
#export
class Hdf2SeqSeq(Transform):
    def __init__(self, seq_inp, seq_out): self.seq_inp,self.seq_out = seq_inp,seq_out
    def encodes(self, o): return (hdf2sequence(o,self.seq_inp),
                                  hdf2sequence(o,self.seq_out))
    def decodes(self, x): return SequenceItem(x)

class Hdf2SeqScal(Transform):
    def __init__(self, seq_inp, scal_out): self.seq_inp,self.scal_out = seq_inp,scal_out
    def encodes(self, o): return (hdf2sequence(o,self.seq_inp),
                                  hdf2scalars(o,self.scal_out))
    def decodes(self, x): return SequenceItem(x) 
class Hdf2SeqScalSeq(Transform):
    def __init__(self, seq_inp,scal_inp, seq_out): self.seq_inp,self.scal_inp,self.seq_out = seq_inp,scal_inp,seq_out
    def encodes(self, o): return (hdf2sequence(o,self.seq_inp),
                                  hdf2scalars(o,self.scal_inp),
                                  hdf2sequence(o,self.seq_out))
    def decodes(self, x): return SequenceItem(x)

class Hdf2SeqScalScal(Transform):
    def __init__(self, seq_inp,scal_inp, scal_out): self.seq_inp,self.scal_inp,self.scal_out = seq_inp,scal_inp,scal_out
    def encodes(self, o): return (hdf2sequence(o,self.seq_inp),
                                  hdf2scalars(o,self.scal_inp),
                                  hdf2scalars(o,self.scal_out))
    def decodes(self, x): return SequenceItem(x) 

In [ ]:
hdf2seq = Pipeline(Hdf2SeqSeq(['current','voltage'],['voltage']))

items = hdf2seq(hdf_files)
len(items),items[0][0].shape

(3, (265598, 2))

### SequenceItem
Damit die Sequenz visualisiert werden kann und auch dritte Informationen gespeichert werden können, wird eine Klasse erstellt 

In [ ]:
#export

#TODO: Fallunterscheidung der Sequenzen
class SequenceItem(Tuple):
    def show(self, ctx=None, **kwargs): 
        plt.figure()
        plt.plot(self[2])

SequenceItem ist nur für die Darstellung eines Tupels von Sequenzen zuständig. Es muss zwischen Skalaren und Vektoriellen Zielgrößen unterschieden werden.

In [ ]:
#export
class SeqTfm(Transform):
    def decodes(self, x): return SequenceItem(x)

SequenceTfm erstellt ein SequenceItem beim decoding für die spätere Darstellung.

## 3. Split in Training, Validation
Splitting kann anhand von vorher bekannten Indizes, dem Dateipfad oder anderen allgemeinen Funktion durchgeführt werden.

Splitting innerhalb einer Sequenzen sollte in der Praxis nur dann geschehen wenn eine einzige Sequenz vorhanden ist. Diese kann dann vorher manuell geteilt werden.


### 3.1 Splitting mit vorgegebenem Index

In [ ]:
splitter = IndexSplitter([1,2])
test_eq(splitter(items),[[0],[1,2]])

### 3.2 Splitting mit allgemeiner Funktion
Items, bei denen die definierte Funktion `True` zurück gibt, werden den Validierungsdatensatz zugeordnet, der Rest dem Training. In diesem Fall wird nach dem Übergeordneten Ordnernamen gesucht.

In [ ]:
splitter = FuncSplitter(lambda o: Path(o).parent.name == 'valid')
test_eq(splitter(hdf_files),[[0,1],[2]])

### 3.3 Splitting anhand des Parent-Folders
Splitter, der Explizit Training und Validierungsordner den Datensätzen zuordnet

In [ ]:
#export
def _parent_idxs(items, name): return mask2idxs(Path(o).parent.name == name for o in items)

def ParentSplitter(train_name='train', valid_name='valid'):
    "Split `items` from the parent folder names (`train_name` and `valid_name`)."
    def _inner(o, **kwargs):
        return _parent_idxs(o, train_name),_parent_idxs(o, valid_name)
    return _inner

In [ ]:
splitter = ParentSplitter()
split_idxs = splitter(hdf_files)
test_eq(split_idxs,[[0,1],[2]])

## 4. Create Datasource

In [ ]:
dsrc = DataSource(items,splits=split_idxs)

In [ ]:
len(dsrc.train),len(dsrc.valid)

(2, 1)

In [ ]:
dsrc.train[0]

((array([[ 0.       ,  4.1873503],
         [-0.0052   ,  4.187454 ],
         [-0.009    ,  4.187548 ],
         ...,
         [ 1.0783   ,  3.7160358],
         [ 1.0739   ,  3.716139 ],
         [ 1.0706   ,  3.7162225]], dtype=float32), array([[4.1873503],
         [4.187454 ],
         [4.187548 ],
         ...,
         [3.7160358],
         [3.716139 ],
         [3.7162225]], dtype=float32)),)

## 5. Create Windows
Aus einer langen Sequenz werden mehrere kurze Sequenzen extrahiert um so verschiedene Teile zu gleicher Zeit dem Model zu zeigen.

Dies geschieht auf Tuple level und ist deshalb kein TupleTransform

In [ ]:
obj = items[0]
len(obj[2]),obj[2]

(64866, array([[3.735982 ],
        [3.7363622],
        [3.7367425],
        ...,
        [2.505974 ],
        [2.502833 ],
        [2.4996917]], dtype=float32))

Testen ob bei vervielfältigung sich der Speicherbedarf massiv ändert

In [ ]:
temp = [obj]*100000

Untersuchung mittels Speicherbedarf bei htop zeigt, dass sich der Speicherbedarf selbst bei 10^5 facher Liste nicht um mehr als 10 MB ändert. Das ist die Darstellungsgrenze

In [ ]:
#export

def createWindows(x):
    win_size = 100
    x_seq = x[0]
    n_win = x_seq.shape[0]//win_size
    win_list = [x]*n_win
    for i,win in enumerate(win_list):
        win_list[i]=(
            win[0][i*win_size:(i+1)*win_size],
            win[1][i*win_size:(i+1)*win_size])
    return L(win_list)

In [ ]:
tst = createWindows(obj)

In [ ]:
tst[1][0].shape

(100, 2)

In [ ]:
pipe_tst = Pipeline(createWindows)
lst = pipe_tst(items)

95.5 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
